In [1]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageOps, ImageTk, ImageEnhance
import customtkinter as ctk
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage import minimum_filter, maximum_filter, median_filter
from skimage.filters import roberts, sobel, scharr, prewitt


# Initialize the root window
root = ctk.CTk()  # Use CTk for the root window
root.geometry("1200x650")
root.title("Image Manipulation Tool")
root.config(bg="#f0f0f0")

pen_color = "black"
pen_size = 5
file_path = ""
image_ref_original = None
image_ref_modified = None

# Store the original and modified images
original_image = None
modified_image = None

# Function to add an image and display it
def add_image():
    global file_path, original_image, image_ref_original, modified_image, image_ref_modified
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        original_image = Image.open(file_path)
        width, height = int(original_image.width / 2), int(original_image.height / 2)
        original_image = original_image.resize((width, height), Image.Resampling.LANCZOS)
        modified_image = original_image.copy()  # Create a copy for manipulation
        image_ref_original = ImageTk.PhotoImage(original_image)
        image_ref_modified = ImageTk.PhotoImage(modified_image)

        canvas_original.config(width=width, height=height)
        canvas_modified.config(width=width, height=height)
        
        canvas_original.create_image(0, 0, image=image_ref_original, anchor="nw")
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")



# Rotate the image
def rotate_image():
    global modified_image, image_ref_modified
    angle = styled_input_dialog("Input", "Enter rotation angle:")
    if angle is not None:
        modified_image = modified_image.rotate(float(angle), expand=True)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.config(width=modified_image.width, height=modified_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")


# Crop the image
def crop_image():
    global modified_image, image_ref_modified
    start_x, start_y = 50, 50  # Sample crop area for demonstration
    end_x, end_y = modified_image.width - 50, modified_image.height - 50
    modified_image = modified_image.crop((start_x, start_y, end_x, end_y))
    image_ref_modified = ImageTk.PhotoImage(modified_image)
    canvas_modified.config(width=modified_image.width, height=modified_image.height)
    canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

#gaussian image
def gaussian_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)
        
        # Define the Gaussian filter
        gaussian_filter = np.array([[1, 4, 7, 4, 1], 
                                    [4, 16, 26, 16, 4], 
                                    [7, 26, 41, 26, 7],
                                    [4, 16, 26, 16, 4],
                                    [1, 4, 7, 4, 1]], dtype=np.float32) / 273
        
        # Apply the Gaussian filter using cv2
        filtered_gaussian_image = cv2.filter2D(array_image, -1, gaussian_filter)
        
        # Convert filtered image back to PIL format
        filtered_image = Image.fromarray(filtered_gaussian_image)
        
        image_ref_modified = ImageTk.PhotoImage(filtered_image)  # Store the reference
        canvas_modified.config(width=filtered_image.width, height=filtered_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

#median filter

def median_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        # Apply the Median filter using cv2
        median_filtered_image = cv2.medianBlur(array_image, 3)
    
        # Convert the filtered image back to PIL format
        median_image = Image.fromarray(median_filtered_image)
    
        image_ref_modified = ImageTk.PhotoImage(median_image)  # Store the reference
        canvas_modified.config(width=median_image.width, height=median_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Roberts filter
def roberts_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        roberts_filtered_image = cv2.filter2D(array_image, -1, kernel)
        
        roberts_image = Image.fromarray(roberts_filtered_image)
        
        image_ref_modified = ImageTk.PhotoImage(roberts_image)  # Store the reference
        canvas_modified.config(width=roberts_image.width, height=roberts_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Sharpen image
def sharpen_image():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        # Define a sharpening kernel
        sharpening_kernel = np.array([[0, -1, 0], 
                                      [-1, 5, -1], 
                                      [0, -1, 0]])
        
        sharpened_image = cv2.filter2D(array_image, -1, sharpening_kernel)
    
        # Convert the sharpened image back to PIL format
        sharpened_image = Image.fromarray(sharpened_image)
    
        image_ref_modified = ImageTk.PhotoImage(sharpened_image)  # Store the reference
        canvas_modified.config(width=sharpened_image.width, height=sharpened_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")
        

# Adjust brightness
def adjust_brightness():
    global modified_image, image_ref_modified
    brightness = styled_input_dialog("Input", "Enter brightness level (0.0 to 10.0):")
    if brightness is not None:
        brightness = float(brightness)
        enhancer = ImageEnhance.Brightness(modified_image)
        modified_image = enhancer.enhance(brightness)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Adjust contrast
def adjust_contrast():
    global modified_image, image_ref_modified
    contrast = styled_input_dialog("Input", "Enter contrast level (0.0 to 10.0):")
    if contrast is not None:
        contrast = float(contrast)
        enhancer = ImageEnhance.Contrast(modified_image)
        modified_image = enhancer.enhance(contrast)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Adjust color balance
def adjust_color_balance():
    global modified_image, image_ref_modified
    color_factor = styled_input_dialog("Input", "Enter color factor (0.0 to 10.0):")
    if color_factor is not None:
        color_factor = float(color_factor)
        enhancer = ImageEnhance.Color(modified_image)
        modified_image = enhancer.enhance(color_factor)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Save the modified image
def save_image():
    if modified_image is not None:
        save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])
        if save_path:
            modified_image.save(save_path)
            messagebox.showinfo("Image Saved", f"Image saved successfully to {save_path}")
    else:
        messagebox.showwarning("No Image", "No modified image to save.")

# Reset the image to its original state
def reset_image():
    global modified_image, image_ref_modified
    modified_image = original_image.copy()
    image_ref_modified = ImageTk.PhotoImage(modified_image)
    canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Style the input dialog with a custom theme
def styled_input_dialog(title, prompt):
    dialog_window = ctk.CTkToplevel(root)  # Use CTkToplevel for the dialog window
    dialog_window.title(title)
    dialog_window.geometry("300x150")
    dialog_window.configure(bg_color="#4A8")  # Set background color for the dialog

    prompt_label = ctk.CTkLabel(dialog_window, text=prompt, font=("Helvetica", 12), text_color="white")
    prompt_label.pack(pady=10)

    input_entry = ctk.CTkEntry(dialog_window, font=("Helvetica", 12))
    input_entry.pack(pady=10)

    def on_submit():
        dialog_window.user_input = input_entry.get()
        dialog_window.destroy()

    submit_button = ctk.CTkButton(dialog_window, text="Submit", command=on_submit, font=("Helvetica", 12), text_color="white", fg_color="#333")
    submit_button.pack(pady=10)

    root.wait_window(dialog_window)

    return getattr(dialog_window, 'user_input', None)



# UI Elements
left_frame = ctk.CTkFrame(root, width=200, height=600, corner_radius=0, bg_color="#333333")  # Use bg_color instead of fg_color
left_frame.pack(side="left", fill="y")

canvas_original = ctk.CTkCanvas(root, width=500, height=600, bg="#ffffff")  # Use standard bg for CTkCanvas
canvas_original.pack(side="left", padx=10, pady=10)

canvas_modified = ctk.CTkCanvas(root, width=500, height=600, bg="#ffffff")  # Use standard bg for CTkCanvas
canvas_modified.pack(side="right", padx=10, pady=10)

# Add image button
add_icon = ctk.CTkImage(dark_image=Image.open("upload.png"), size=(20, 20))
image_button = ctk.CTkButton(left_frame, text="Add Image", command=add_image, image=add_icon)
image_button.pack(pady=20)



# Rotate image button
rotate_icon = ctk.CTkImage(dark_image=Image.open("rotate.png"), size=(20, 20))
rotate_button = ctk.CTkButton(left_frame, text="Rotate Image", command=rotate_image, image=rotate_icon)
rotate_button.pack(pady=10)




# Crop image button
crop_icon = ctk.CTkImage(dark_image=Image.open("crop.png"), size=(20, 20))
crop_button = ctk.CTkButton(left_frame, text="Crop Image", command=crop_image, image=crop_icon)
crop_button.pack(pady=10)



# Adjust brightness button
brightness_icon = ctk.CTkImage(dark_image=Image.open("brightness.png"), size=(20, 20))
brightness_button = ctk.CTkButton(left_frame, text="Adjust Brightness", command=adjust_brightness, image=brightness_icon)
brightness_button.pack(pady=10)

#gaussian filter
gaussian_icon = ctk.CTkImage(dark_image=Image.open("gaussian.png"), size=(20, 20))
gaussian_filter_button = ctk.CTkButton(left_frame, text="Gaussian Filter", command=gaussian_filter, image=gaussian_icon)
gaussian_filter_button.pack(pady=10)

#median filter
median_icon = ctk.CTkImage(dark_image=Image.open("graph.png"), size=(20, 20))
median_filter_button =ctk.CTkButton(left_frame, text="Median Blur", command=median_filter, image=median_icon)
median_filter_button.pack(pady=10)

#roberts filter
roberts_filter_button =ctk.CTkButton(left_frame, text="Roberts Image", command=roberts_filter)
roberts_filter_button.pack(pady=10)

#sharpen image
sharpen_icon = ctk.CTkImage(dark_image=Image.open("sharpen.png"), size=(20, 20))
sharpen_button =ctk.CTkButton(left_frame, text="Sharpen Image", command=sharpen_image,image=sharpen_icon)
sharpen_button.pack(pady=10)

# Adjust contrast button
contrast_icon = ctk.CTkImage(dark_image=Image.open("contrast.png"), size=(20, 20))
contrast_button = ctk.CTkButton(left_frame, text="Adjust Contrast", command=adjust_contrast, image=contrast_icon)
contrast_button.pack(pady=10)

# Adjust color balance button
filter_icon = ctk.CTkImage(dark_image=Image.open("filter.png"), size=(20, 20))
color_balance_button = ctk.CTkButton(left_frame, text="Adjust Color Balance", command=adjust_color_balance, image=filter_icon)
color_balance_button.pack(pady=10)

# Reset image button
reset_icon = ctk.CTkImage(dark_image=Image.open("undo.png"), size=(20, 20))
reset_button = ctk.CTkButton(left_frame, text="Reset Image", command=reset_image, image=reset_icon)
reset_button.pack(pady=20)

# Save image button
save_icon = ctk.CTkImage(dark_image=Image.open("save.png"), size=(20, 20))
save_button = ctk.CTkButton(left_frame, text="Save Image", command=save_image, image=save_icon)
save_button.pack(pady=20)



root.mainloop()

